In [1]:
# This allows importing Jupyter notebooks as modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import JupyterNotebookImporter

In [2]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import pprint
import random
import datetime
import math
import bisect
import time
from os import listdir
from os.path import isfile, join

from IPython.display import clear_output
from AlgoPlotting import XYChart
from AlgoPlotting import CandlestickChart

importing Jupyter notebook from AlgoPlotting.ipynb


In [3]:
def get_minute_data_dir():
    return r'''Data\KibotData\SP500_1'''

def get_single_day_data_dir():
    return r'''Data\SingleDayData\SP500'''

def get_day_data_dir():
    return r'''Data\KibotData\SP500_day'''

In [4]:
# Maybe move this inside data library to keep everything like this in one place
def load_minute_data(instrument, date):
    datestr = date.strftime('_%Y%m%d')
    file_name = instrument + datestr + '.txt'
    single_day_data_dir = get_single_day_data_dir()
    single_day_file_path = single_day_data_dir + '\\' + file_name
    # Try loading single day file of minute data which takes less than 6 years
    try:
        col_names = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'DateTime']
        single_day_data = pd.read_csv(single_day_file_path, names=col_names)
        date_times = (single_day_data['Date']+single_day_data['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
        single_day_data['DateTime'] = list(date_times)
        #print(single_day_data)
        #print(single_day_data['Open'])
    # Cry and load the full minute data file, then save only the day's worth of data from it to a better file
    except FileNotFoundError:
        single_day_data = load_raw_minute_data(instrument, date, single_day_file_path)
    return single_day_data

# Lazy name
def load_raw_minute_data(instrument, date, single_day_file_path):
    minute_data_dir = get_minute_data_dir()
    col_names = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    minute_data = pd.read_csv(join(minute_data_dir, instrument+'.txt'), names=col_names)
    date_times = (minute_data['Date']+minute_data['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
    minute_data['DateTime'] = list(date_times)
    minute_data = minute_data[[(dt >= datetime.datetime(date.year, date.month, date.day, 9, 30, 0)) and (dt <= datetime.datetime(date.year, date.month, date.day, 16, 0, 0)) for dt in minute_data['DateTime']]].copy()
    # No headers or index in the file, want it in the same format as the full minute data files
    minute_data.to_csv(single_day_file_path, header=False, index=False)
    return minute_data

In [5]:
class DataLibrary:
    def __init__(self, filename):
        self.filename = filename
        #self.df = pd.DataFrame()
        self.df = pd.read_csv('DataLibrary/' + self.filename, index_col = 0)
        
    def get_data(self, date, instrument, data_name):
        key = self.keygen(date, instrument)
        try:
            data = self.df[data_name].loc[key]
        except:
            self.regen(date, instrument)
            data = self.df[data_name].loc[key]
        
        if not data or np.isnan(data):
            self.regen(date, instrument)
            data = self.df[data_name].loc[key]
        
        return data
    
    def regen(self, date, instrument):
        key = self.keygen(date, instrument)
        data_names = ['Open', 'High', 'Low', 'Close', 'Volume', 'FirstMinClose']
        
        minutedatadir = get_minute_data_dir()
        minutecolnames = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
        pricedata = pd.read_csv(join(minutedatadir, instrument+'.txt'), names=minutecolnames)
        pricedata['DateTime'] = (pricedata['Date']+pricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
        pricedata = pricedata[[(dt >= datetime.datetime(date.year, date.month, date.day, 9, 30, 0)) and (dt <= datetime.datetime(date.year, date.month, date.day, 16, 0, 0)) for dt in pricedata['DateTime']]].copy()

        for data_name in data_names:
            try:
                temp = self.df[data_name]
            except:
                self.df[data_name] = [None] * len(self.df)

        try:
            temp = self.df.loc[key]
        except:
            new_row = {}
            for data_name in data_names:
                new_row[data_name] = None

            to_append = pd.DataFrame(new_row, index=[key])
            self.df = self.df.append(to_append)

        self.df['Open'][key] = pricedata['Open'].iloc[0]
        self.df['Close'][key] = pricedata['Close'].iloc[-1]
        self.df['High'][key] = np.max(pricedata['High'])
        self.df['Low'][key] = np.min(pricedata['Low'])
        self.df['Volume'][key] = np.sum(pricedata['Volume'])
        self.df['FirstMinClose'][key] = pricedata['Close'].iloc[0]
        
    def remove_col(self, data_name):
        self.df = self.df.drop(columns=[data_name])
        
    def keygen(self, date, instrument):
        return str(date) + str(instrument)
        
    def close(self):
        timestr = time.strftime("_%Y%m%d-%H%M%S")
        self.df.to_csv('DataLibrary/' + self.filename)
        self.df.to_csv('DataLibrary/Backups/' + self.filename[:-4] + timestr + '.txt')

In [6]:
# dl = DataLibrary('data_library.txt')
# #dl.remove_col('Date')
# dl.get_data(datetime.date(2018,1,4), 'SHLD', 'Volume')
# dl.get_data(datetime.date(2018,1,5), 'AAPL', 'Volume')
# dl.close()

In [7]:
daydatadir = get_day_data_dir()
files = [f for f in listdir(daydatadir) if isfile(join(daydatadir, f))]
instruments = [f[0:-4] for f in files]

colnames = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
daypricedata = {}
for f in instruments:
    df = pd.read_csv(join(daydatadir, f+'.txt'), names=colnames)
    df['Date'] = df['Date'].map(lambda x: datetime.date(int(x[6:10]), int(x[0:2]), int(x[3:5])))
    df['Gap'] = np.insert(np.array(df['Open'][1:])-np.array(df['Close'][:-1]),0,0)
    df = df.set_index('Date')
    daypricedata[f] = df

In [8]:
def get_gap_instruments(param_dict):
    start_date = param_dict['Start Date']
    end_date = param_dict['End Date']
    dollar_volume_threshold = param_dict['Dollar Volume Threshold']
    min_gap_threshold = param_dict['Minimum Gap Threshold']
    
    num_instruments = 1
    max_gap_instruments = {}
    gaps = {}

    for ii in range((end_date - start_date).days):
        date = start_date + datetime.timedelta(days=ii)
        max_abs_gap = 0
        max_gap_instrument = None
        for instrument in instruments:
            if date in daypricedata[instrument].index:
                gap = daypricedata[instrument].loc[date]['Gap']/daypricedata[instrument].loc[date]['Open']
                abs_gap = abs(gap)
                # 5 dollars is the currently accepted threshold for penny stock classification
                dollar_volume = daypricedata[instrument].loc[date]['Volume'] * daypricedata[instrument].loc[date]['Open']
                # above_price_filter = daypricedata[instrument].loc[date]['Open'] > 5.00
                above_dollar_volume_threshold = dollar_volume > dollar_volume_threshold
                above_min_gap_threshold = abs_gap > min_gap_threshold
#                 if abs_gap > max_abs_gap and not above_dollar_volume_threshold:
#                     print(instrument, ' did not pass the dollar volume. Value: ', dollar_volume)
#                 if abs_gap > max_abs_gap and not above_min_gap_threshold:
#                     print(instrument, ' did not pass the minimum gap threshold. Value: ', gap)
                if abs_gap > max_abs_gap and above_dollar_volume_threshold and above_min_gap_threshold:
                    max_gap = gap
                    max_abs_gap = abs_gap
                    max_gap_instrument = instrument
        if max_gap_instrument:
            max_gap_instruments[date] = max_gap_instrument
            gaps[date] = max_gap
            print(date, max_gap_instrument, max_gap)
            
    return max_gap_instruments, gaps

In [9]:
# idx = [ii for ii in range(len(max_gap_instruments))]
# random.shuffle(idx)

# rand_dates = list(max_gap_instruments.keys())
# rand_instruments = list(max_gap_instruments.values())

# for ii in range(len(rand_dates)):
#     max_gap_instruments[rand_dates[ii]] = rand_instruments[idx[ii]]

In [10]:
def day_gap_trading_strategy(portfolio, data_dict, param_dict):
    minutedatadir = get_minute_data_dir()
    minutecolnames = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    
    max_gap_instruments = data_dict['max_gap_instruments']
    gaps = data_dict['gaps']
    
    start_date = param_dict['Start Date']
    end_date = param_dict['End Date']
    dl = DataLibrary('data_library.txt')

    for ii in range((end_date - start_date).days):
        date = start_date + datetime.timedelta(days=ii)
        
        try:
            instrument = max_gap_instruments[date]
            gap = gaps[date]
        except KeyError:
            instrument = None
            
        if instrument:
#             try:
                
#                 pricedata = dl.get_data(datetime.date(2018,1,4), instrument, 'Open')
#                 print(date, instrument)
#             except:
#                 continue

            try:
                open_price = dl.get_data(date, instrument, 'Open')
                close_price = dl.get_data(date, instrument, 'Close')
                first_min_close_price = dl.get_data(date, instrument, 'FirstMinClose')
                transaction_price = dl.get_data(date, instrument, 'FirstMinClose')
                open_transaction_dt = datetime.datetime(date.year, date.month, date.day, 9, 31, 0)
                close_transaction_dt = datetime.datetime(date.year, date.month, date.day, 15, 59, 0)
            except:
                continue
                
            initial_net_value = portfolio.net_value
            if first_min_close_price > open_price:
            #if np.sign(gap) < 0:
                quantity = np.floor(10000/transaction_price)
                portfolio.buy(instrument, quantity, transaction_price, open_transaction_dt)
                portfolio.sell(instrument, quantity, close_price, close_transaction_dt)
            elif first_min_close_price < open_price:
            #elif np.sign(gap) > 0:
                quantity = np.floor(10000/transaction_price)
                portfolio.sell(instrument, quantity, transaction_price, open_transaction_dt)
                portfolio.buy(instrument, quantity, close_price, close_transaction_dt)
            daily_profit = portfolio.net_value - initial_net_value
#             portfolio.daily_history.append(
#                 {
#                     'Date': date,
#                     'Instrument': instrument,
#                     'Profit': daily_profit
#                 }
#             )
            #portfolio.update(instrument, row)
    
    portfolio.portfolio_history = pd.DataFrame(portfolio.portfolio_history)
    portfolio.transaction_history = pd.DataFrame(portfolio.transaction_history)
    dl.close()

In [11]:
def gap_trading_strategy(portfolio, data_dict, param_dict):
    
    max_gap_instruments = data_dict['max_gap_instruments']
    gaps = data_dict['gaps']
    
    start_date = param_dict['Start Date']
    end_date = param_dict['End Date']
    sell_target = param_dict['Sell Target']
    stop_loss = param_dict['Stop Loss']
    max_trades = param_dict['Max Trades']
    
    for ii in range((end_date - start_date).days):
        date = start_date + datetime.timedelta(days=ii)
        
        try:
            instrument = max_gap_instruments[date]
            gap = gaps[date]
        except KeyError:
            instrument = None
            
        if instrument:
            try:
                minute_data = load_minute_data(instrument, date)
                portfolio.positions[instrument] = 0
                open_price = minute_data['Open'].iloc[0]
                #close_price = dl.get_data(date, instrument, 'Close')
            except:
                continue
            
            if not minute_data.index.empty and len(minute_data) > 1:
                portfolio.daily_trades = 0
                for index in range(len(minute_data)):
                    row = minute_data.iloc[index]

                    #if index == len(minute_data)-1 or (np.sign(row['Close'] - open_price) != np.sign(portfolio.positions[instrument]) and index != 0):
                    #if index == len(minute_data)-1 or row['Close'] < stop_loss or row['Close'] > sell_target:
                    if index == len(minute_data)-1:
                        if portfolio.positions[instrument] < 0:
                            portfolio.buy(instrument, -portfolio.positions[instrument], row['Close'], row['DateTime'])
                        elif portfolio.positions[instrument] > 0:
                            portfolio.sell(instrument, portfolio.positions[instrument], row['Close'], row['DateTime'])
                    
                    elif index == 0:
                        if row['Close'] > open_price:
                            break
                        #if np.sign(gap) < 0:
                            if portfolio.positions[instrument] <= 0:
                                quantity = np.floor(10000/row['Close'])
                                portfolio.buy(instrument, -portfolio.positions[instrument] + quantity, row['Close'], row['DateTime'])
                        elif row['Close'] < open_price:
                        #elif np.sign(gap) > 0:
                            if portfolio.positions[instrument] >= 0:
                                quantity = np.floor(10000/row['Close'])
                                portfolio.sell(instrument, portfolio.positions[instrument] + quantity, row['Close'], row['DateTime'])
                    
                    elif row['Close'] > open_price:
                        if portfolio.positions[instrument] <= 0:
                            quantity = np.floor(10000/row['Close'])
                            if portfolio.daily_trades == max_trades - 1:
                                quantity = 0
                            portfolio.buy(instrument, -portfolio.positions[instrument] + quantity, row['Close'], row['DateTime'])
                    
                    elif row['Close'] < open_price:
                        if portfolio.positions[instrument] >= 0:
                            quantity = np.floor(10000/row['Close'])
                            if portfolio.daily_trades == max_trades - 1:
                                quantity = 0
                            portfolio.sell(instrument, portfolio.positions[instrument] + quantity, row['Close'], row['DateTime'])

                    if portfolio.daily_trades == max_trades:
                        break
                        
                    #portfolio.update(instrument, row)
    
    portfolio.portfolio_history = pd.DataFrame(portfolio.portfolio_history)
    portfolio.transaction_history = pd.DataFrame(portfolio.transaction_history)                

In [12]:
def bear_mode_gap_trading_strategy(portfolio, data_dict, param_dict):
    
    max_gap_instruments = data_dict['max_gap_instruments']
    gaps = data_dict['gaps']
    
    start_date = param_dict['Start Date']
    end_date = param_dict['End Date']
    sell_target = param_dict['Sell Target']
    stop_loss = param_dict['Stop Loss']
    max_trades = param_dict['Max Trades']
    
    for ii in range((end_date - start_date).days):
        date = start_date + datetime.timedelta(days=ii)
        
        try:
            instrument = max_gap_instruments[date]
            gap = gaps[date]
        except KeyError:
            instrument = None
            
        if instrument:
            try:
                minute_data = load_minute_data(instrument, date)
                portfolio.positions[instrument] = 0
                open_price = minute_data['Open'].iloc[0]
                #close_price = dl.get_data(date, instrument, 'Close')
            except:
                continue
            
            if not minute_data.index.empty and len(minute_data) > 1:
                portfolio.daily_trades = 0
                stop_loss = open_price
                for index in range(len(minute_data)):
                    row = minute_data.iloc[index]
                    
                    if index == 0:
                        if row['Close'] > open_price:
                            break
                        elif row['Close'] < open_price:
                            if portfolio.positions[instrument] >= 0:
                                quantity = np.floor(10000/row['Close'])
                                portfolio.sell(instrument, portfolio.positions[instrument] + quantity, row['Close'], row['DateTime'])
                    
                    elif index == len(minute_data)-1 or row['High'] > stop_loss:
                        if portfolio.positions[instrument] < 0:
                            if row['High'] > stop_loss and False:
                                sell_price = stop_loss
                            else:
                                sell_price = row['Close']
                            portfolio.buy(instrument, -portfolio.positions[instrument], sell_price, row['DateTime'])
                            
                    # This try catch prevented a crash "can't multiply sequence with non-int float", but nothing
                    # was printed. Probably empty, should ensure that doesn't screw anything up further down.
                    try:
                        stop_loss = min(stop_loss, 1.03*row['Close'])
                    except:
                         print("Ghost at: ", instrument, " ", date, " row['Close']: ", row['Close'])
                    
    
    portfolio.portfolio_history = pd.DataFrame(portfolio.portfolio_history)
    portfolio.transaction_history = pd.DataFrame(portfolio.transaction_history)                

In [13]:
def bull_mode_gap_trading_strategy(portfolio, data_dict, param_dict):
    
    max_gap_instruments = data_dict['max_gap_instruments']
    gaps = data_dict['gaps']
    
    start_date = param_dict['Start Date']
    end_date = param_dict['End Date']
    sell_target = param_dict['Sell Target']
    stop_loss = param_dict['Stop Loss']
    max_trades = param_dict['Max Trades']
    
    for ii in range((end_date - start_date).days):
        date = start_date + datetime.timedelta(days=ii)
        
        try:
            instrument = max_gap_instruments[date]
            gap = gaps[date]
        except KeyError:
            instrument = None
            
        if instrument:
            try:
                minute_data = load_minute_data(instrument, date)
                portfolio.positions[instrument] = 0
                open_price = minute_data['Open'].iloc[0]
                #close_price = dl.get_data(date, instrument, 'Close')
            except:
                continue
            
            if not minute_data.index.empty and len(minute_data) > 1:
                portfolio.daily_trades = 0
                stop_loss = open_price
                for index in range(len(minute_data)):
                    row = minute_data.iloc[index]
                    
                    if index == 0:
                        if row['Close'] < open_price:
                            break
                        elif row['Close'] > open_price:
                            if portfolio.positions[instrument] <= 0:
                                quantity = np.floor(10000/row['Close'])
                                portfolio.buy(instrument, -portfolio.positions[instrument] + quantity, row['Close'], row['DateTime'])
                    
                    elif index == len(minute_data)-1 or row['Low'] < stop_loss:
                        if portfolio.positions[instrument] > 0:
                            if row['Low'] < stop_loss and False:
                                sell_price = stop_loss
                            else:
                                sell_price = row['Close']
                            portfolio.sell(instrument, portfolio.positions[instrument], sell_price, row['DateTime'])
                            
                    # This try catch prevented a crash "can't multiply sequence with non-int float", but nothing
                    # was printed. Probably empty, should ensure that doesn't screw anything up further down.
                    try:
                        stop_loss = max(stop_loss, 0.97*row['Close'])
                    except:
                         print("Ghost at: ", instrument, " ", date, " row['Close']: ", row['Close'])
                    
    
    portfolio.portfolio_history = pd.DataFrame(portfolio.portfolio_history)
    portfolio.transaction_history = pd.DataFrame(portfolio.transaction_history)                

In [14]:
class Portfolio:
    def __init__(self, param_dict):
        
        self.param_dict = param_dict
        self.starting_cash = param_dict['Starting Cash']
        self.cash = self.starting_cash
        self.positions = {}
        self.commission = param_dict['Commission']
        
        self.net_value = 0
        self.portfolio_history = []
        self.transaction_history = []
        self.daily_history = []
        self.total_commission_losses = 0
        
        # A bit gross but want daily history inside portfolio - need to monitor date to tell when days are over
        self.current_date = None
        self.current_date_initial_net_value = None
        self.current_instrument = None
        self.daily_trades = 0
        
    def update(self, instrument, price, date_time):
        new_net_value = self.cash + self.positions[instrument] * price
        # Fill the daily_history dict with the previous day's values if this is a new day
        current_date = date_time.date()
        if self.current_date and self.current_date != current_date:
            # self.net_value not updated yet, so this is ok
            daily_profit = self.net_value - self.current_date_initial_net_value
            self.daily_history.append(
                {
                    'Date': self.current_date,
                    'Instrument': self.current_instrument,
                    'Profit': daily_profit
                }
            )
            if daily_profit < -400:
                print("Big loss of " + str(daily_profit) + " on " + str(self.current_date) + ", " + str(self.current_instrument))
            elif daily_profit > 400:
                print("Big win of " + str(daily_profit) + " on " + str(self.current_date) + ", " + str(self.current_instrument))
            
            self.current_date = current_date
            self.current_date_initial_net_value = new_net_value
            self.current_instrument = instrument
        elif self.current_date is None:
            self.current_date = current_date
            self.current_instrument = instrument
            self.current_date_initial_net_value = new_net_value
        
        self.net_value = new_net_value
            
        self.portfolio_history.append(
            {
                "DateTime": date_time,
                "Cash": self.cash,
                "Position": self.positions,
                "Net Value": self.net_value
            }
        )
        
    def buy(self, instrument, quantity, price, date_time):
            
        self.cash -= quantity*price + self.commission
        self.total_commission_losses += self.commission

        if not instrument in self.positions.keys():
            self.positions[instrument] = 0
        self.positions[instrument] += quantity

        self.cash = round(self.cash*100)/100

        self.transaction_history.append(
            {
                "DateTime": date_time,
                "Date": date_time.date(),
                "Instrument" : instrument,
                "Price": price,
                "Quantity": quantity
            }
        )
        self.daily_trades += 1
        self.update(instrument, price, date_time)
        
    def sell(self, instrument, quantity, price, date_time):
        
        self.cash += quantity*price - self.commission
        self.total_commission_losses += self.commission

        if not instrument in self.positions.keys():
            self.positions[instrument] = 0
        self.positions[instrument] -= quantity

        self.cash = round(self.cash*100)/100

        self.transaction_history.append(
            {
                "DateTime": date_time,
                "Date": date_time.date(),
                "Instrument" : instrument,
                "Price": price,
                "Quantity": -quantity
            }
        )
        self.daily_trades += 1
        self.update(instrument, price, date_time)
        
    def logging_dict(self):
        max_net_value = self.portfolio_history["Net Value"][0]
        max_drawdown = 0
        max_drawdown_percentage = 0
        for ii in range(len(self.portfolio_history)):
            max_net_value = max(max_net_value, self.portfolio_history["Net Value"][ii])
            max_drawdown = max(max_drawdown, max_net_value - self.portfolio_history["Net Value"][ii])
            max_drawdown_percentage = max(max_drawdown_percentage, 1 - self.portfolio_history["Net Value"][ii] / max_net_value)
            
        d = {
            'param_dict' : self.param_dict,
            'Starting Cash' : self.starting_cash,
            'Commission' : self.commission,
            'Profit' : self.net_value-self.starting_cash,
            'Annualized Return (Compounding)' : (self.net_value / self.starting_cash)**(365 / (param_dict['End Date']-param_dict['Start Date']).days) - 1,
            'Annualized Profit (Linear)' : (self.net_value - self.starting_cash)*(365 / (param_dict['End Date']-param_dict['Start Date']).days),
            'Number of Trades' : len(self.transaction_history),
            'Max Drawdown' : max_drawdown,
            'Max Drawdown %' : max_drawdown / max_net_value * 100
        }
        
        return d

In [15]:
def log(log_file_path, param_dict, portfolio, notes):
    log_file = open(log_file_path, 'w')
    pp = pprint.PrettyPrinter(indent=4, stream=log_file)
    pp.pprint(portfolio.logging_dict())
    pp.pprint({'Notes' : notes})
    log_file.flush()
    log_file.close()

In [16]:
timestr = time.strftime("%Y%m%d-%H%M%S")
log_path = 'logs/AlgoTrading1/'
log_file_name = timestr + ".txt"
log_file_path = log_path + log_file_name
#notes = input("Enter notes for this run: ")
notes = ''

# For gap_trading_strategy2
param_dict = {
    'Starting Cash' : 50000,
    'Commission' : 5,
    'Start Date': datetime.date(2018,1,1),
    'End Date': datetime.date(2018,3,1),
    'Dollar Volume Threshold': 25e6,
    'Minimum Gap Threshold': 0.05
}

# For gap_trading_strategy3
param_dict = {
    'Starting Cash' : 50000,
    'Commission' : 5,
    'Start Date': datetime.date(2017,1,1),
    'End Date': datetime.date(2017,2,1),
    'Dollar Volume Threshold': 25e6,
    'Minimum Gap Threshold': 0.05,
    'Max Trades': 2,
    'Stop Loss': 0.1,
    'Sell Target': 0.1
}

In [17]:
max_gap_instruments, gaps = get_gap_instruments(param_dict)
data_dict = {
    'max_gap_instruments': max_gap_instruments,
    'gaps': gaps
}

2017-01-03 XRX 0.11536842105263151
2017-01-04 THC 0.05582822085889571
2017-01-05 KSS -0.19159706403442167
2017-01-06 REGN -0.05692410119840208
2017-01-09 AYI -0.13933672234325828
2017-01-10 ILMN 0.12440457779152497
2017-01-11 SIG -0.07192052980132459
2017-01-13 GME -0.11031320775562081
2017-01-17 NRG 0.05530104712041878
2017-01-18 NTRS -0.05527072652244594
2017-01-19 CSX 0.14674043781573254
2017-01-20 BMY -0.10950149126544527
2017-01-23 QCOM -0.07161854103343457
2017-01-24 ATI 0.12699228791773773
2017-01-25 STX 0.11821342604974597
2017-01-26 MAT -0.15737203972498093
2017-01-27 JNPR -0.10386879730866269
2017-01-30 VAR -0.13773222293401666
2017-01-31 UAA -0.37809523809523815


In [18]:
portfolio = Portfolio(param_dict)
bear_mode_gap_trading_strategy(portfolio, data_dict, param_dict)
#bull_mode_gap_trading_strategy(portfolio, data_dict, param_dict)

log(log_file_path, param_dict, portfolio, notes)

Big loss of -436.4599999999991 on 2017-01-03, XRX


In [19]:
class ProfitCharts():
    def __init__(self, portfolio):
        self.portfolio = portfolio
        cash = portfolio.portfolio_history['Cash']
        net_value = portfolio.portfolio_history['Net Value']
        
        x_vals = [dt.strftime("%y/%m/%d, %H:%M:%S") for dt in portfolio.portfolio_history['DateTime']]
        tick_vals = [ii for ii in range(len(x_vals)) if 
                                 portfolio.portfolio_history['DateTime'][ii].time() == datetime.time(9, 30)]
        xaxisdatetimes = [portfolio.portfolio_history['DateTime'][ii] for ii in tick_vals]
        tick_text = [dt.strftime(' %y/%m/%d ') if dt.time() == datetime.time(9, 30)
                                 else dt.strftime(' %H:%M ') for dt in xaxisdatetimes]

        data1 = [ dict(
                type = 'scatter',
                x = x_vals,
                y = cash,
                name = 'Cash'),

                dict(
                type = 'scatter',
                x = x_vals,
                y = net_value - cash,
                name = 'Position'),

                dict(
                type = 'scatter',
                x = x_vals,
                y = net_value,
                name = 'Net Value')
               ]
        
        layout1 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'DateTime'
                    ),
                    type = 'category', 
                    categoryorder = 'category ascending',
                    tickvals = tick_vals,
                    ticktext = tick_text
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Value'
                    )
                ))
        
        self.chart1 = go.FigureWidget( data=data1, layout=layout1)
        
        data2 = [
                dict(
                type = 'scatter',
                x = x_vals,
                y = [nv - net_value[0] for nv in net_value],
                name = 'Profit')
               ]
        
        layout2 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'DateTime'
                    ),
                    type = 'category', 
                    categoryorder = 'category ascending',
                    tickvals = tick_vals,
                    ticktext = tick_text
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Profit'
                    )
                ))
        
        self.chart2 = go.FigureWidget( data=data2, layout=layout2)        
        # DEFINE DISPLAY CONFIGURATION
        self.display = ipywidgets.VBox([self.chart1, self.chart2])

In [20]:
profit_charts = ProfitCharts(portfolio)
profit_charts.display

    'data': [{'name': 'Cash',
              'type': 'scatter',
              'ui…

In [21]:
# Takes daily history from portfolio class as input. It's a dict like {'Date': datetime.date, 'Instrument': 'AAPL', 'Profit': 5000} 
class TradingStrategyViewer:
    
    def __init__(self, portfolio, filter_function, load_all=False):
        self.portfolio = portfolio
        self.daily_history = self.portfolio.daily_history
        self.transaction_history = self.portfolio.transaction_history.set_index('Date')
        self.filter_function = filter_function
        self.load_all = load_all
        self.index = 0
        self.last_entry = None
        
        #self.xy_chart = XYChart()
        self.xy_chart = CandlestickChart()
        
        self.filtered_history = [d for d in self.daily_history if self.filter_function(d)]
        self.filtered_transactions = {}
        self.start()
    
    # Jupyter doesn't seem to play nice with keypress listeners/msvcrt, so we can use this terrible clear_output/input solution
    def start(self):
        if len(self.filtered_history) != 0:
            # Process/write to file all the data beforehand if specified
            if self.load_all:
                self.load_all_data()
            while self.last_entry != 'q':
                clear_output(wait=False)
                history = self.filtered_history[self.index]
                minute_data = load_minute_data(history['Instrument'], history['Date'])
                #x = minute_data['DateTime']
                open_price = list(minute_data['Open'])
                close_price = list(minute_data['Close'])
                day_open_price = [open_price[0] for _ in open_price]
                
                #y = close_price # Otherwise bitches about key error 0
                ohlc = [list(minute_data['Open']),
                       list(minute_data['High']),
                       list(minute_data['Low']),
                       list(minute_data['Close'])]
                
                title = history['Instrument'] + ', ' + history['Date'].strftime('%d %m %Y') + ', Index = ' + str(self.index)
                transactions = self.transaction_history.loc[history['Date']]
                
                #self.xy_chart.update(y=[y, day_open_price], title=title)
                self.xy_chart.update(ohlc=ohlc, title=title)
                
                print(len(transactions.index.values))
                for idx in range(len(transactions)):
                    transaction = transactions.iloc[idx]
                    #print(transaction)
                    print(str(transaction['DateTime']))
                    #plot_idx = list(minute_data['DateTime']).index(str(transaction['DateTime']))
                    plot_idx = list(minute_data['DateTime']).index(transaction['DateTime'])
                    
                    if transaction['Quantity'] > 0:
                        self.xy_chart.add_buy_marker(plot_idx, transaction['Price'])
                    else:
                        self.xy_chart.add_sell_marker(plot_idx, transaction['Price'])
                prompt = 'Hit enter for next plot, . + Enter for last plot, q + Enter to quit.'
                self.xy_chart.display()
                self.last_entry = input(prompt)
                if self.last_entry == '.':
                    self.index = (self.index - 1) % len(self.filtered_history)
                else:
                    self.index = (self.index + 1) % len(self.filtered_history)
        else:
            clear_output(wait=False)
            print("No days matching the given filter!")
                
    def load_all_data(self):
        for idx, history in enumerate(self.filtered_history):
            percent = (idx/len(self.filtered_history))*100
            percent_str = f'{percent:.2f}% done loading single day data'
            print(percent_str, end='\r')
            load_minute_data(history['Instrument'], history['Date'])
            

In [22]:
def good_day_filter(daily_history):
    if daily_history['Profit'] > 100:
        return True
    else:
        return False

In [23]:
def bad_day_filter(daily_history):
    if daily_history['Profit'] < -101:
        return True
    else:
        return False

In [24]:
#tr = TradingStrategyViewer(portfolio.daily_history, good_day_filter)
tr = TradingStrategyViewer(portfolio, bad_day_filter, load_all=True)

2
2017-01-18 09:30:00
2017-01-18 09:32:00


FigureWidget({
    'data': [{'close': [79.28, 79.68, 80.41, ..., 80.51, 80.4, 80.4],
              'high': [81…

Hit enter for next plot, . + Enter for last plot, q + Enter to quit.q


In [25]:
tr = TradingStrategyViewer(portfolio, good_day_filter, load_all=True)

2
2017-01-09 09:30:00
2017-01-09 10:13:00


FigureWidget({
    'data': [{'close': [204.95, 203.05, 199.04, ..., 200.42, 200.48, 200.77],
              'hi…

Hit enter for next plot, . + Enter for last plot, q + Enter to quit.q


In [26]:
profits = [p['Profit'] for p in portfolio.daily_history]

In [27]:
class Histo:
    def __init__(self):
        data = [ dict(
                type = 'histogram',
                x = profits)
               ]
        
        layout = dict(
                title = 'Hist'
        )
        
        self.display = go.FigureWidget( data=data, layout=layout)
        
Histo().display

FigureWidget({
    'data': [{'type': 'histogram',
              'uid': '8c6f4742-2f70-418c-88f3-311c2d305696',…

In [28]:
portfolio.transaction_history

,Date,DateTime,Instrument,Price,Quantity
0,2017-01-03,2017-01-03 09:30:00,XRX,23.640,-423.0
1,2017-01-03,2017-01-03 09:31:00,XRX,24.660,423.0
2,2017-01-04,2017-01-04 09:30:00,THC,16.180,-618.0
3,2017-01-04,2017-01-04 09:55:00,THC,16.210,618.0
4,2017-01-05,2017-01-05 09:30:00,KSS,39.360,-254.0
5,2017-01-05,2017-01-05 09:31:00,KSS,39.500,254.0
6,2017-01-09,2017-01-09 09:30:00,AYI,204.950,-48.0
7,2017-01-09,2017-01-09 10:13:00,AYI,197.240,48.0
8,2017-01-17,2017-01-17 09:30:00,NRG,15.092,-662.0
9,2017-01-17,2017-01-17 09:41:00,NRG,15.241,662.0
